# LangChain MCP 클라이언트를 활용한 문서 기반 QA 시스템

## 강의 개요

이번 장에서는 LangChain의 MCP(Model Context Protocol) 클라이언트를 사용하여 문서 기반 질문-답변 시스템을 구축하는 방법을 학습합니다.

## 학습 목표

- MCP 클라이언트의 기본 개념 이해
- 문서 업로드 및 처리 과정 이해
- 비동기 프로그래밍을 활용한 QA 시스템 구현

### 1. 라이브러리 임포트

- `asyncio`: 비동기 프로그래밍을 위한 모듈
- `MultiServerMCPClient`: 여러 MCP 서버와 통신할 수 있는 클라이언트
- `json`: JSON 형태의 응답 처리
- `os`: 파일 경로 처리

In [6]:
import asyncio
from langchain_mcp_adapters.client import MultiServerMCPClient
import json
import os

### 2. MCP 도구 호출 함수

**주요 기능:**
- 클라이언트에서 사용 가능한 도구 목록 조회
- 지정된 도구 검색 및 실행


In [7]:
async def call_mcp_tool(client, tool_name, tool_args):
    tools = client.get_tools()
    tool = next((t for t in tools if t.name == tool_name), None)
    if tool is None:
        raise ValueError(f"Tool '{tool_name}' not found")
    return await tool.coroutine(**tool_args)

### 3. 문서 업로드 함수

**핵심 개념:**
- 파일 경로를 인자로 받아 문서 업로드
- JSON 형태의 결과 파싱

In [8]:
async def upload_document(client, file_path):
    result = await call_mcp_tool(client, "upload_document", {"file_path": file_path})
    parsed_result = json.loads(result[0])
    print("[문서 업로드 결과]", parsed_result)

### 4. 질문-답변 함수 

**핵심 개념:**
- 사용자가 입력한 질의를 MCP 서버로 전송하여 응답 수신
- 튜플 형태의 응답을 처리하여 텍스트만 추출
- 사용자 친화적인 출력 형식으로 질문과 답변을 구분하여 표시


In [9]:
async def ask_question(client, query):
    ask_result = await call_mcp_tool(client, "ask_question", {"query": query})
    
    # 결과가 튜플인 경우 첫 번째 요소만 추출
    if isinstance(ask_result, tuple):
        response_text = ask_result[0]
    else:
        response_text = ask_result
    
    # 응답 텍스트 출력 (깔끔하게 정리)
    print("\n" + "="*50)
    print(f"질문: {query}")
    print("="*50)
    print(response_text)
    print("="*50 + "\n")

### 5. 메인 실행 함수

**구성 요소:**
- **클라이언트 설정**: 로컬 서버 연결 정보
- **전송 방식**: SSE(Server-Sent Events) 사용
- **문서 처리**: 절대 경로로 PDF 파일 업로드
- **질문 실행**: 업로드된 문서 기반 질의

In [11]:
async def main():
    async with MultiServerMCPClient(
        {
            "qa": {
                "url": "http://localhost:8005/sse",
                "transport": "sse",
            }
        }
    ) as client:
        pdf_path = os.path.abspath("./document/news_weather.pdf")        
        await upload_document(client, pdf_path)
        await ask_question(client, "집중 호우의 원인은 무엇인가요?")
        
try:
    asyncio.get_event_loop().run_until_complete(main())
except RuntimeError:  
    asyncio.ensure_future(main())

C:\Users\IISLab\AppData\Local\Temp\ipykernel_34556\3665780568.py:18: RuntimeWarning: coroutine 'main' was never awaited
  asyncio.ensure_future(main())


[문서 업로드 결과] {'status': '문서 저장 완료'}

질문: 집중 호우의 원인은 무엇인가요?
집중 호우의 주요 원인은 다음과 같습니다:

1. **저기압 시스템**: 기상청과 장은철 교수는 이번 집중호우의 직접적인 원인으로 서해에서 유입된 저기압이 장마전선과 만나 많은 비를 가져왔다고 분석했습니다. 특히, 강력하게 발달한 저기압이 북태평양 고기압과 부딪혀 많은 강수량을 발생시켰습니다.

2. **대기 순환 패턴의 변화**: 민승기 교수와 장은철 교수는 해수 온도 상승이 대기 순환 패턴에 영향을 미치고 있다고 보고 있습니다. 서해의 온난화가 빠르게 진행되면서 대기의 수증기 함유량 증가와 강수 패턴 변화를 초래할 수 있습니다.

3. **엘니뇨 현상**: 전 지구 해수면 온도 상승으로 인한 엘니뇨 현상도 기후 변화에 영향을 미쳐 강수 패턴에 변화를 가져올 수 있습니다.

이러한 요인들이 복합적으로 작용하여 짧은 시간 내에 많은 비가 내리는 집중 호우 현상을 초래하고 있습니다.



### 6. 도구 목록확인

In [24]:
async def discover_available_tools(client):
    tools = client.get_tools()
    
    print("서버에서 사용 가능한 도구 목록")
    print("="*60)
    
    for i, tool in enumerate(tools, 1):
        print(f"{i}. 도구명: {tool.name}")
        print("-" * 40)
    
   
    print("="*60 + "\n")
    
    return [tool.name for tool in tools]

In [25]:

async def main():
    async with MultiServerMCPClient(
        {
            "qa": {
                "url": "http://localhost:8005/sse",
                "transport": "sse",
            }
        }
    ) as client:
        
        # 1단계: 사용 가능한 도구 목록 확인
        await discover_available_tools(client)

# 실행
await main()

서버에서 사용 가능한 도구 목록
1. 도구명: upload_document
----------------------------------------
2. 도구명: ask_question
----------------------------------------
3. 도구명: list_documents
----------------------------------------



### 실습: 위의 예제에서 사용하지 않은 도구를 사용한 결과를 출력하세요


1. 도구명: upload_document
----------------------------------------
2. 도구명: ask_question
----------------------------------------
3. 도구명: list_documents

In [1]:
async def show_document_list(client):
    
    
    #빈칸에 들어갈 것만 채워주세요
    result = await call_mcp_tool(client, "___________", {}) #빈칸에 들어갈 것만 채워주세요
    
    if isinstance(result, tuple):
        response_text = result[0]
    else:
        response_text = result
    
    print(" 문서 목록:")
    print(response_text)

In [27]:
# 5. 메인 실행
async def main():
    async with MultiServerMCPClient(
        {
            "qa": {
                "url": "http://localhost:8005/sse",
                "transport": "sse",
            }
        }
    ) as client:
        
        #실습 pdf 파일 추가 로드 
        pdf_path = os.path.abspath("./document/원자력안전관리_설명자료.pdf")
        await upload_document(client, pdf_path)
        await show_document_list(client)

# 실행
await main()

[문서 업로드 결과] {'status': '문서 저장 완료'}
📋 문서 목록:
 총 2개의 문서가 저장되어 있습니다:

  📄 1. news_weather.pdf
  📄 2. 원자력안전관리_설명자료.pdf
